In [7]:
from time import time

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import time
import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms
import tensorflow as tf
import random as r
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook
import copy

import Config
import Dataloader as DL

import matplotlib.pyplot as plt
import pandas as pd
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC

In [3]:
datasets=[
    "KDD Cup 1999",                            #0
    "Microsoft Challenge BIG 2015"             #1
]
presets = {
    "KDD Cup 1999": {
        "NeuralHD": [300,2,3,.1],
        "OnlineHD": [300,1.0,.1,30,True],
        "MLP": [100,5,.001],
        "SVM": [10000]
    },
    "Microsoft Challenge BIG 2015": {
        "NeuralHD": [3000,6,10,.1],
        "OnlineHD": [3000,1.0,.1,30,True],
        "MLP": [100,30,.001],
        "SVM": [None]
    }
}
def normalized(x,y):
    xtrain, x_test, ytrain, y_test = None,None,None,None
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y, shuffle=True)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()
    return x.numpy(), x_test.numpy(), y.numpy(), y_test.numpy(), scaler
def getuniquevalues(columnname,df):
    values={}
    i=0
    for entry in df[columnname]:
        if entry not in values:
            values[entry]=i
            i+=1
    return values
def get_dataset(name):
    if name==datasets[0]:
        path="../../Data/"
        attacks_types = {
            'normal': 'normal','back': 'dos','buffer_overflow': 'u2r','ftp_write': 'r2l','guess_passwd': 'r2l',
        'imap': 'r2l','ipsweep': 'probe','land': 'dos','loadmodule': 'u2r','multihop': 'r2l','neptune': 'dos',
        'nmap': 'probe','perl': 'u2r','phf': 'r2l','pod': 'dos','portsweep': 'probe','rootkit': 'u2r','satan': 'probe',
        'smurf': 'dos','spy': 'r2l','teardrop': 'dos','warezclient': 'r2l','warezmaster': 'r2l',
        }
        cols ="""duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,
        urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,
        num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,
        count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,
        diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,
        dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,
        dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate"""
        
        columns =[]
        for c in cols.split(','):
            if(c.strip()):
                columns.append(c.strip())
        print(len(columns))
        columns.append('target')
        print(len(columns))

        attack_categories=["dos","u2r","r2l",'probe','normal']
        df = pd.read_csv(path+"kddcup.data_10_percent.gz", names = columns)
        df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
        del df['target']
        df.head()
        num_cols = df._get_numeric_data().columns
        
        cate_cols = list(set(df.columns)-set(num_cols))
        cate_cols.remove('Attack Type')
        for col in cate_cols:
            df[col]=df[col].map(getuniquevalues(col,df))
        data=df.to_numpy()
        Y=df['Attack Type'].map(getuniquevalues('Attack Type',df))
        Y=Y.to_numpy()
        X=data[:,:-1]
        print(Y.shape)
        print(X.shape)
        print(getuniquevalues('Attack Type',df))
        xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)
    if name==datasets[1]:
        path="../../Data/malware-classification/"
        map={}
        mapping=pd.read_csv(path + "trainLabels.csv")
        Y=mapping["Class"].to_numpy()
        for i in range(0,len(Y)):
            map[mapping["Id"][i]]=mapping["Class"][i]-1
        byte_features=pd.read_csv(path+"result.csv")
        byte_features['ID']  = byte_features['ID'].str.split('.').str[0]
        byte_features.head(3)
        byte_features['ID']=byte_features['ID'].map(map)
        data=byte_features.to_numpy()
        X=data[:,1:]
        Y=data[:,0]
        xtrain, x_test, ytrain, y_test,scaler= normalized(X,Y)
    return xtrain,x_test,ytrain,y_test

datasetname="Microsoft Challenge BIG 2015"

xtrain,x_test,ytrain,y_test=get_dataset(datasetname)


In [56]:
class NeuralHD:
    def __init__(self, classes : int, features : int, dim : int = 400):
        #Configure for hdb, hdc, and hde classes
        self.param=Config.config
        self.param['nClasses'] = classes
        self.param['nFeatures']= features
        #hypervector size
        self.param['D']=dim
        #encoder
        self.hde=None
        #classifier
        self.hdc=None
    def __call__(self, x : torch.Tensor):
        #True iff the model has been trained
        assert self.hde!=None and self.hdc!=None
        #return predicted values
        return self.predict(x)
    def predict(self,x):
        #Get hypervectors for all data points
        trainencoded=self.hde.encodeData(x)
        #return predictions based on similarity to classification hypervectors
        return np.array(self.hdc.predict(trainencoded))
    def fit(self,traindata, trainlabels,valdata, vallabels,
                   epochs,
                   regenloops,  # list of effective dimensions to reach 
                   percentDrop # drop/regen rate 
                    ):
        
        # Initialize basis & classifier
        hdb = HDB.HD_basis(HDB.Generator.Vanilla, self.param)
        # store generated basis
        basis = hdb.getBasis()
        # updata params after basis generation
        self.param = hdb.getParam()
        # make encoder based on basis
        self.hde = HDE.HD_encoder(basis)
        # find encoded training vectors
        trainencoded = self.hde.encodeData(traindata)
        # print(valdata)
        valencoded = self.hde.encodeData(valdata)
        # Initialize classification hypervectors
        self.hdc = HDC.HD_classifier(self.param["D"], self.param["nClasses"], 0)

        # calculate amount of dropped dimensions based on percent and original dimension
        amountDrop = int(percentDrop * self.hdc.D)#self.param.D?
        # print("Updating times:", regenloops)

        for i in range(regenloops+1): # For each eDs to reach, will checkpoints
            # print("regenloop: " + str(i))
            # train for x epochs
            perfect=self.trainreploop(epochs,trainencoded,trainlabels, valencoded,vallabels)
            #if its the last regeneration training, stop before doing another dimension drop; stop if 100% accuracy
            if i==regenloops or perfect:
                return #self.hdc,self.hde - unnecessary now that hdc and hde are within a class
            # print("regeneration")
            #do the dimension drop and regeneration
            trainencoded=self.regen(hdb,amountDrop,traindata)
        return "error","error"
    
    def trainreploop(self,epochs,trainencoded,trainlabels, valencoded, vallabels):
        # Do the train 
        for j in range(epochs):
            # do one pass of training
            train_acc = 100 * self.hdc.fit(trainencoded, trainlabels, self.param)
            # print(self.hdc.D)
            # print(valencoded)
            val_acc = 100 * self.hdc.test(valencoded, vallabels)
            #Test accuracy used to be in here but I took it out because we dont usually know it
            print("Train: %.2f \t \t Test: %.2f"%(train_acc,val_acc))
            # If accuracy is 100, finish
            if train_acc == 100:
                return True
        return False
    def regen(self,hdb,amountDrop,traindata):
        #make order of dimensions according to variaince; also store variances <-- unnecessary?
        var, orders = self.hdc.evaluateBasis()
        #drop dimensions with lowest variance
        toDrop = orders[:amountDrop]
        # print("Variances stats: max %.2f, min %.2f, mean %.2f"%(max(var),min(var),np.mean(var)))
        #update basis by randomizing dimension
        hdb.updateBasis(toDrop)
        # move the new basis into the encoder
        self.hde.updateBasis(hdb.basis)
        # normalize previous classes so retrain has enough effect
        self.hdc.updateClasses()
        # get new encoded training data
        return self.hde.encodeData(traindata)    

In [31]:
model=NeuralHD(9,xtrain.shape[1],3000)

In [32]:
model.fit(xtrain,ytrain,30,5,.1)

Updating times: 5
Updating basis......
Updating basis......
Updating basis......
Updating basis......
Updating basis......


In [28]:
def get_class_accuracy_breakdown(model,x_test,y_test, output=False):
    acc=[]
    points=[]
    try:
        for i in range (0,len(np.unique(y_test))):
            yhat= model.predict(x_test[y_test==i])
            if len(yhat.shape)==2:
                yhat=np.array([row.argmax() for row in yhat])
            acc.append((yhat==i).mean())
            points.append(len(yhat))
    except:
        for i in range (0,len(np.unique(y_test))):
            yhat= model.predict(torch.from_numpy(x_test[y_test==i]))
            acc.append((yhat==i).float().mean())
            points.append(len(yhat))
    # print(yhat[:30])
    totacc=sum([acc[i]*points[i] for i in range(0,len(acc))])/sum(points)
    if output:
        plt.bar(range(0,len(acc)),acc,color=np.random.rand(3,))
        plt.title("Accuracy Total: " + str(totacc))
        plt.show()
    return totacc

In [66]:
accuracies=[]
for i in range(0,20):
    model=NeuralHD(len(np.unique(y_test)),xtrain.shape[1],3000)
    model.fit(xtrain[:7000],ytrain[:7000],xtrain[7000:],ytrain[7000:],5,5,.1)
    accuracies.append(get_class_accuracy_breakdown(model,x_test,y_test))


Train: 66.69 	 	 Test: 74.98
Train: 74.67 	 	 Test: 84.71
Train: 77.81 	 	 Test: 64.12
Train: 79.14 	 	 Test: 85.75
Train: 80.73 	 	 Test: 86.01
Updating basis......
Train: 74.90 	 	 Test: 51.35
Train: 78.60 	 	 Test: 70.98
Train: 81.34 	 	 Test: 86.88
Train: 82.04 	 	 Test: 66.12
Train: 83.33 	 	 Test: 48.48
Updating basis......
Train: 76.09 	 	 Test: 84.62
Train: 78.04 	 	 Test: 82.54
Train: 81.30 	 	 Test: 85.84
Train: 82.79 	 	 Test: 81.06
Train: 82.56 	 	 Test: 85.58
Updating basis......
Train: 75.39 	 	 Test: 76.89
Train: 79.41 	 	 Test: 82.62
Train: 81.46 	 	 Test: 79.67
Train: 83.69 	 	 Test: 53.61
Train: 83.40 	 	 Test: 84.19
Updating basis......
Train: 77.24 	 	 Test: 66.12
Train: 80.23 	 	 Test: 87.14
Train: 81.06 	 	 Test: 64.99
Train: 82.63 	 	 Test: 86.71
Train: 85.13 	 	 Test: 89.75
Updating basis......
Train: 76.44 	 	 Test: 80.89
Train: 80.33 	 	 Test: 48.57
Train: 82.39 	 	 Test: 66.90
Train: 83.50 	 	 Test: 82.62
Train: 84.97 	 	 Test: 50.91
Train: 66.46 	 	 Test: 59

In [67]:
print(accuracies)

[0.8001472211998528, 0.9109311740890689, 0.8376886271623114, 0.8439455281560545, 0.9138755980861244, 0.8693411851306588, 0.8840633051159367, 0.860875966139124, 0.9112992270887008, 0.6481413323518587, 0.8306956201693044, 0.881854987118145, 0.9020979020979021, 0.8575634891424365, 0.8078763341921237, 0.5752668384247331, 0.8730217151269782, 0.84688995215311, 0.8689731321310269, 0.8726536621273463]


In [68]:
print(sum(accuracies)/len(accuracies))

0.8398601398601396


In [27]:
accuracies=[]
points=[]
for i in range (0,model.param['nClasses']):
    yhat= model(x_test[y_test==i])
    acc = (yhat==i).mean()
    accuracies.append(acc)
    points.append(len(yhat))
    print('class '+str(i)+' accuracy: ' +str(acc))
    print('points: '+ str(len(yhat)))
print(sum([accuracies[i]*points[i] for i in range(0,len(accuracies))])/sum(points))

class 0 accuracy: 0.9074550128534704
points: 389
class 1 accuracy: 0.9683042789223455
points: 631
class 2 accuracy: 0.9090909090909091
points: 737
class 3 accuracy: 0.9754098360655737
points: 122
class 4 accuracy: 0.0
points: 11
class 5 accuracy: 0.7918781725888325
points: 197
class 6 accuracy: 0.979381443298969
points: 97
class 7 accuracy: 0.684887459807074
points: 311
class 8 accuracy: 0.918918918918919
points: 222
0.8910563121089436
